### Global Total Average and Refined Spatial Autocorrelation
#### This notebook grabs the 14 year average salinity measurements that I calculated in my Lab 3 notebook and creates a new layer not for web mapping. It then calculates a Moran's I to prove further analysis is needed to describe river input influences on ocean salinity.

In [5]:
import arcpy
import os
from arcpy.sa import *
from arcpy.stats import *
import matplotlib.pyplot as plt

# Setup
arcpy.CheckOutExtension("Spatial")
arcpy.env.overwriteOutput = True
arcpy.env.workspace = r"C:\Users\Liliana\Documents\ArcGIS\Projects\SalinityDrafting\Salinity_Analysis.gdb"
print("Environment ready")

Environment ready


#### Section 1: Load 14 Year Global Average

In [6]:
# The original global mean
global_mean = "Global_Mean_Salinity_2011_2024"

In [7]:
# Get statistics
desc = arcpy.Describe(global_mean)
print(f"  Extent: {desc.extent}")
print(f"  Cell size: {desc.meanCellWidth}°")

  Extent: -180 -90 180 90 NaN NaN NaN NaN
  Cell size: 0.25°


In [8]:
# Calculate global statistics
min_val = arcpy.management.GetRasterProperties(global_mean, "MINIMUM")
max_val = arcpy.management.GetRasterProperties(global_mean, "MAXIMUM")
mean_val = arcpy.management.GetRasterProperties(global_mean, "MEAN")
    
print(f"  Min salinity: {float(min_val.getOutput(0)):.1f} PSU")
print(f"  Max salinity: {float(max_val.getOutput(0)):.1f} PSU")
print(f"  Mean salinity: {float(mean_val.getOutput(0)):.1f} PSU")

  Min salinity: 5.3 PSU
  Max salinity: 39.7 PSU
  Mean salinity: 34.3 PSU


### Section 2: Viz for Story Map

In [9]:
# Export to a clean layer
output_layer = "Global_Salinity_StoryMap"

In [10]:
# Copy the raster
arcpy.management.CopyRaster(global_mean, output_layer)

print(f"✓ Created visualization layer: {output_layer}")

✓ Created visualization layer: Global_Salinity_StoryMap


In [11]:
# Export to format for StoryMap
output_tiff = r"C:\Users\Liliana\Desktop\Global_Salinity_StoryMap.tif"
arcpy.management.CopyRaster(
    output_layer,
    output_tiff,
    pixel_type="32_BIT_FLOAT",
    nodata_value="-9999"
)

print(f"✓ Exported to: {output_tiff}")
print("  This file is ready for upload")

✓ Exported to: C:\Users\Liliana\Desktop\Global_Salinity_StoryMap.tif
  This file is ready for upload


### Section 3: Spatial Autocorrelation

#### Setup

In [16]:
import arcpy
from arcpy.stats import *
import json
import os
import matplotlib.pyplot as plt
import numpy as np

# Environment setup
arcpy.env.overwriteOutput = True
gdb_path = r"C:\Users\Liliana\Documents\ArcGIS\Projects\SalinityDrafting\Salinity_Analysis.gdb"
arcpy.env.workspace = gdb_path

# Check for raster
salinity_raster = "Global_Mean_Salinity_2011_2024"
print(f"✓ Dataset loaded: {salinity_raster}")

✓ Dataset loaded: Global_Mean_Salinity_2011_2024


### Data sampling

In [17]:
# Create point sample from raster
sample_points = "Global_Salinity_Sample_Points"

print("Converting raster to points...")
arcpy.conversion.RasterToPoint(salinity_raster, sample_points, "Value")

# Get sample statistics
point_count = int(arcpy.management.GetCount(sample_points).getOutput(0))
print(f"✓ Created {point_count:,} sample points")

Converting raster to points...
✓ Created 604,513 sample points


### Global Moran's I

In [21]:
print("Calculating spatial autocorrelation...")
moran_result = arcpy.stats.SpatialAutocorrelation(
    sample_points,          # Input feature class
    "grid_code",          # Field containing salinity values
    "GENERATE_REPORT"     # Generate detailed output
)

# Get tool execution messages
messages = moran_result.getMessages()

Calculating spatial autocorrelation...


### Extracting results

In [29]:
import re

messages = moran_result.getMessages()

# Extract using regular expressions
morans_i_match = re.search(r'"Moran\'s Index", "([0-9.]+)"', messages)
z_score_match = re.search(r'"z-score", "([0-9.]+)"', messages)
p_value_match = re.search(r'"p-value", "([0-9.]+)"', messages)

if morans_i_match and z_score_match and p_value_match:
    print(f"Moran's I: {morans_i_match.group(1)}")
    print(f"Z-score:  {z_score_match.group(1)}")
    print(f"P-value:  {p_value_match.group(1)}")
else:
    print(messages)

Moran's I: 0.998186
Z-score:  2830.438100
P-value:  0.000000
